# Running TCAV


This notebook walks you through things you need to run TCAV. In high level, you need:

1. **example images in each folder**
 * images for each concept
 * images for the class/labels of interest
 * random images that will be negative examples when learning CAVs (images that probably don't belong to any concepts)
2. **model wrapper**: an instance of  ModelWrapper abstract class (in model.py). This tells TCAV class (tcav.py) how to communicate with your model (e.g., getting internal tensors)
3. **act_generator**: an instance of ActivationGeneratorInterface that tells TCAV class how to load example data and how to get activations from the model



## Requirements

    pip install the tcav and tensorflow packages (or tensorflow-gpu if using GPU)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tcav.activation_generator as act_gen
import tcav.cav as cav
import tcav.model  as model
import tcav.tcav as tcav
import tcav.utils as utils
import tcav.utils_plot as utils_plot # utils_plot requires matplotlib
import os 
import tensorflow as tf

/Users/justina/anaconda3/envs/py37_tf113/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/justina/anaconda3/envs/py37_tf113/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/justina/anaconda3/envs/py37_tf113/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/justina/anaconda3/env

## Step 1. Store concept and target class images to local folders

and tell TCAV where they are.

**source_dir**: where images of concepts, target class and random images (negative samples when learning CAVs) live. Each should be a sub-folder within this directory.

Note that random image directories can be in any name. In this example, we are using `random500_0`, `random500_1`,.. for an arbitrary reason. 


You need roughly 50-200 images per concept and target class (10-20 pictures also tend to work, but 200 is pretty safe).


**cav_dir**: directory to store CAVs (`None` if you don't want to store)

**target, concept**: names of the target class (that you want to investigate) and concepts (strings) - these are folder names in source_dir

**bottlenecks**: list of bottleneck names (intermediate layers in your model) that you want to use for TCAV. These names are defined in the model wrapper below.



In [ ]:
# folder prefix 
# Mac
prefix = '/Users/justina/Documents/EPFL/thesis/project/hnsc/tcav'
# Ubuntu
# prefix = '/usr/local/google/home/beenkim'

print ('REMEMBER TO UPDATE FOLDER PREFIX!')

# This is the name of your model wrapper (InceptionV3 and GoogleNet are provided in model.py)
model_to_run = 'GoogleNet'  
user = 'justina'
# the name of the parent directory that results are stored (only if you want to cache)
project_name = 'tcav_class_test'
working_dir = prefix + "/tmp/" + user + '/' + project_name
print(working_dir)
# where activations are stored (only if your act_gen_wrapper does so)
activation_dir =  working_dir+ '/activations/'
print(activation_dir)
# where CAVs are stored. 
# You can say None if you don't wish to store any.
cav_dir = working_dir + '/cavs/'
print(cav_dir)
# where the images live. 
source_dir = prefix + "/source_dir/"
print(source_dir)
bottlenecks = [ 'mixed4c']  # @param 
      
utils.make_dir_if_not_exists(activation_dir)
utils.make_dir_if_not_exists(working_dir)
utils.make_dir_if_not_exists(cav_dir)

# this is a regularizer penalty parameter for linear classifier to get CAVs. 
alphas = [0.1]   

target = 'zebra'  
concepts = ["dotted","striped","zigzagged"]   


## Step 2. Write your model wrapper

Next step is to tell TCAV how to communicate with your model. See `model.GoolgeNetWrapper_public ` for details.

You can define a subclass of ModelWrapper abstract class to do this. Let me walk you thru what each function does (tho they are pretty self-explanatory).  This wrapper includes a lot of the functions that you already have, for example, `get_prediction`.

### 1. Tensors from the graph: bottleneck tensors and ends
First, store your bottleneck tensors in `self.bottlenecks_tensors` as a dictionary. You only need bottlenecks that you are interested in running TCAV with. Similarly, fill in `self.ends` dictionary with `input`, `logit` and `prediction` tensors.

### 2. Define loss
Get your loss tensor, and assigned it to `self.loss`. This is what TCAV uses to take directional derivatives. 

While doing so, you would also want to set 
```python
self.y_input 
```
this simply is a tensorflow place holder for the target index in the logit layer (e.g., 0 index for a dog, 1 for a cat).
For multi-class classification, typically something like this works:

```python
self.y_input = tf.placeholder(tf.int64, shape=[None])
```

For example, for a multiclass classifier, something like below would work. 

```python
    # Construct gradient ops.
    with g.as_default():
      self.y_input = tf.placeholder(tf.int64, shape=[None])

      self.pred = tf.expand_dims(self.ends['prediction'][0], 0)

      self.loss = tf.reduce_mean(
          tf.nn.softmax_cross_entropy_with_logits(
              labels=tf.one_hot(self.y_input, len(self.labels)),
              logits=self.pred))
    self._make_gradient_tensors()
```

### 3. Call _make_gradient_tensors in __init__() of your wrapper
```python
_make_gradient_tensors()  
```
does what you expect - given the loss and bottleneck tensors defined above, it adds gradient tensors.

### 4. Fill in labels, image shapes and a model name.
Get the mapping from labels (strings) to indice in the logit layer (int) in a dictionary format.

```python
def id_to_label(self, idx)
def label_to_id(self, label)
```

Set your input image shape at  `self.image_shape`


Set your model name to `self.model_name`

You are done with writing the model wrapper! I wrote two model wrapers, InceptionV3 and Googlenet.


**sess**: a tensorflow session.

In [8]:
sess = utils.create_session()

# GRAPH_PATH is where the trained model is stored.
# GRAPH_PATH =  prefix + "/tensorflow_inception_graph.pb"
GRAPH_PATH = "./frozen_model.pb"
# LABEL_PATH is where the labels are stored. Each line contains one class, and they are ordered with respect to their index in 
# the logit layer. (yes, id_to_label function in the model wrapper reads from this file.)
# For example, imagenet_comp_graph_label_strings.txt looks like:
# dummy                                                                                      
# kit fox
# English setter
# Siberian husky ...

# LABEL_PATH = prefix + "/imagenet_comp_graph_label_strings.txt"
LABEL_PATH = "./model/labels.txt"

mymodel = model.XceptionHPVWrapper(sess,
                                        GRAPH_PATH,
                                        LABEL_PATH)

# mymodel = model.GoolgeNetWrapper_public(sess,
#                                         GRAPH_PATH,
#                                         LABEL_PATH)

BOTTLENECK: {'add/add': <tf.Tensor 'import/xception_2/add/add:0' shape=(?, ?, ?, 128) dtype=float32>, 'add_1/add': <tf.Tensor 'import/xception_2/add_1/add:0' shape=(?, ?, ?, 256) dtype=float32>, 'add_2/add': <tf.Tensor 'import/xception_2/add_2/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_3/add': <tf.Tensor 'import/xception_2/add_3/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_4/add': <tf.Tensor 'import/xception_2/add_4/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_5/add': <tf.Tensor 'import/xception_2/add_5/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_6/add': <tf.Tensor 'import/xception_2/add_6/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_7/add': <tf.Tensor 'import/xception_2/add_7/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_8/add': <tf.Tensor 'import/xception_2/add_8/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_9/add': <tf.Tensor 'import/xception_2/add_9/add:0' shape=(?, ?, ?, 728) dtype=float32>, 'add_10/add': <tf.Tensor 'import/xception_2/add_10/add:0' sha

## Step 3. Implement a class that returns activations (maybe with caching!)

Lastly, you will implement a class of the ActivationGenerationInterface which TCAV uses to load example data for a given concept or target, call into your model wrapper and return activations. I pulled out this logic outside of mymodel because this step often takes the longest. By making it modular, you can cache your activations and/or parallelize your computations, as I have done in `ActivationGeneratorBase.process_and_load_activations` in `activation_generator.py`.


The `process_and_load_activations` method of the activation generator must return a dictionary of activations that has concept or target name as  a first key, and the bottleneck name as a second key. So something like:

```python
{concept1: {bottleneck1: [[0.2, 0.1, ....]]},
concept2: {bottleneck1: [[0.1, 0.02, ....]]},
target1: {bottleneck1: [[0.02, 0.99, ....]]}
```


In [ ]:
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=100)

## You are ready to run TCAV!

Let's do it.

**num_random_exp**: number of experiments to confirm meaningful concept direction. TCAV will search for this many folders named `random500_0`, `random500_1`, etc. You can alternatively set the `random_concepts` keyword to be a list of folders of random concepts. Run at least 10-20 for meaningful tests. 

**random_counterpart**: as well as the above, you can optionally supply a single folder with random images as the "positive set" for statistical testing. Reduces computation time at the cost of less reliable random TCAV scores. 


In [ ]:
tf.logging.set_verbosity(0)
## only running num_random_exp = 10 to save some time. The paper number are reported for 500 random runs. 
mytcav = tcav.TCAV(sess,
                   target,
                   concepts,
                   bottlenecks,
                   act_generator,
                   alphas,
                   cav_dir=cav_dir,
                   num_random_exp=10)
print ('This may take a while... Go get coffee!')
results = mytcav.run(run_parallel=False)
print ('done!')

In [ ]:
utils_plot.plot_results(results, num_random_exp=10)

In [ ]:
# from keras.applications.inception_v3 import InceptionV3
# from keras.models import Model
# from keras import backend as K
# base_model = InceptionV3(weights='imagenet')

# base_model.layers


In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile
sess = utils.create_session()

f = gfile.FastGFile("./tensorflow_inception_graph.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
f.close()

sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)

In [ ]:
tf.layers

In [ ]:
graph = tf.get_default_graph()
LOGDIR='./logs/tests/2/'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)
# graph.get_operations()
for op in graph.get_operations():
    print("name: {}".format(op.name))
    print("values: {}".format(op.values()))

In [ ]:
# from scipy.stats import ttest_ind
# import numpy as np
# import tensorflow as tf
# from tcav.tcav_results.results_pb2 import Result, Results
# import os
# import keras
# from keras import backend as K
# tf.keras.backend.set_learning_phase(0)


# def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
#     """
#     Freezes the state of a session into a pruned computation graph.

#     Creates a new computation graph where variable nodes are replaced by
#     constants taking their current value in the session. The new graph will be
#     pruned so subgraphs that are not necessary to compute the requested
#     outputs are removed.
#     @param session The TensorFlow session to be frozen.
#     @param keep_var_names A list of variable names that should not be frozen,
#                           or None to freeze all the variables in the graph.
#     @param output_names Names of the relevant graph outputs.
#     @param clear_devices Remove the device directives from the graph for better portability.
#     @return The frozen graph definition.
#     """
#     from tensorflow.python.framework.graph_util import convert_variables_to_constants
#     graph = session.graph
#     with graph.as_default():
#         freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
#         output_names = output_names or []
#         output_names += [v.op.name for v in tf.global_variables()]
#         # Graph -> GraphDef ProtoBuf
#         input_graph_def = graph.as_graph_def()
#         if clear_devices:
#             for node in input_graph_def.node:
#                 node.device = ""
#         frozen_graph = convert_variables_to_constants(session, input_graph_def,
#                                                       output_names, freeze_var_names)
#         return frozen_graph

# def write_pb(keras_model_path):
#     model = tf.keras.models.load_model(keras_model_path, compile=False)
#     model.compile(loss='sparse_categorical_crossentropy',
#                 optimizer=tf.keras.optimizers.Adam())
#     frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])
#     tf.train.write_graph(frozen_graph, "model", "hpv_xception.pb", as_text=False)    


In [ ]:
# model_path = "/Users/justina/Documents/EPFL/thesis/project/hnsc/trained_model.h5"
# write_pb(model_path)

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import graph_io
from tensorflow.keras.applications.inception_v3 import InceptionV3


def freeze_graph(graph, session, output):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, ".", "frozen_model.pb", as_text=True)

tf.keras.backend.set_learning_phase(0) # this line most important

keras_model_path = "/Users/justina/Documents/EPFL/thesis/project/hnsc/trained_model.h5"

base_model = tf.keras.models.load_model(keras_model_path, compile=False)
base_model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam())
session = tf.keras.backend.get_session()

INPUT_NODE = base_model.inputs[0].op.name
OUTPUT_NODE = base_model.outputs[0].op.name
freeze_graph(session.graph, session, [out.op.name for out in base_model.outputs])

In [4]:
from tensorflow.python.platform import gfile
sess = utils.create_session()

f = gfile.FastGFile("./model/frozen_model.pb", 'rb')
graph_def = tf.GraphDef()
# Parses a serialized binary message into the current message.
graph_def.ParseFromString(f.read())
print(graph_def.node)
f.close()

sess.graph.as_default()
# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
tf.import_graph_def(graph_def)
g = tf.get_default_graph()
g.get_operations()


-> 323     graph_def = tf.GraphDef.FromString(tf.gfile.Open(saved_path, 'rb').read())
    324 
    325     with tf.name_scope(scope) as sc:

DecodeError: Error parsing message

Instructions for updating:
Use tf.gfile.GFile.


/Users/justina/anaconda3/envs/py37_tf113/lib/python3.7/site-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[<tf.Operation 'import/block1_conv1_2/kernel' type=Const>,
 <tf.Operation 'import/block1_conv1_bn_2/gamma' type=Const>,
 <tf.Operation 'import/block1_conv1_bn_2/beta' type=Const>,
 <tf.Operation 'import/block1_conv1_bn_2/moving_mean' type=Const>,
 <tf.Operation 'import/block1_conv1_bn_2/moving_variance' type=Const>,
 <tf.Operation 'import/block1_conv2_2/kernel' type=Const>,
 <tf.Operation 'import/block1_conv2_bn_2/gamma' type=Const>,
 <tf.Operation 'import/block1_conv2_bn_2/beta' type=Const>,
 <tf.Operation 'import/block1_conv2_bn_2/moving_mean' type=Const>,
 <tf.Operation 'import/block1_conv2_bn_2/moving_variance' type=Const>,
 <tf.Operation 'import/block2_sepconv1_2/depthwise_kernel' type=Const>,
 <tf.Operation 'import/block2_sepconv1_2/pointwise_kernel' type=Const>,
 <tf.Operation 'import/block2_sepconv1_bn_2/gamma' type=Const>,
 <tf.Operation 'import/block2_sepconv1_bn_2/beta' type=Const>,
 <tf.Operation 'import/block2_sepconv1_bn_2/moving_mean' type=Const>,
 <tf.Operation 'import/

In [ ]:
print(graph_def.node)

In [ ]:
# base_model.layers

g.get_operations()

In [ ]:
for op in g.get_operations():
    print("name: {}".format(op.name))
    print("values: {}".format(op.values()))


In [ ]:
LOGDIR='./logs/tests/1/'
train_writer = tf.summary.FileWriter(LOGDIR)
train_writer.add_graph(sess.graph)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/tests/1/


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/tests/2/